# Imports

In [3]:
%pdb off
%load_ext autoreload
%autoreload 2
import sys
import importlib
from pathlib import Path

import numpy as np

# required to enable non-blocking interaction:
# %gui qt
# !env QT_API="pyqt5"
%gui qt5
# %gui qt6
# from PyQt5.Qt import QApplication
# # start qt event loop
# _instance = QApplication.instance()
# if not _instance:
#     _instance = QApplication([])
# app = _instance

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd


# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

enable_saving_to_disk = False

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer

from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs, find_local_session_paths, batch_load_session

def build_eloy_computation_configs(sess, **kwargs):
    """ OPTIONALLY can be overriden by implementors to provide specific filter functions """
    # (4.0, 4.0)cm bins, (6.0, 6.0)cm gaussian smoothing
    # peak frate > 2Hz 
    # return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(6.0, 6.0), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None), spike_analysis=None)]
    # return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(2.5, 2.5), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None), spike_analysis=None)]
    # return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(0.2, 0.2), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None), spike_analysis=None)]
    return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(0.2, 0.2), frate_thresh=0.2, time_bin_size=0.025, computation_epochs = None), spike_analysis=None)]



# global_data_root_parent_path = Path(r'W:\Data') # Windows Apogee
global_data_root_parent_path = Path(r'/media/MAX/Data') # Diba Lab Workstation Linux
# global_data_root_parent_path = Path(r'/Volumes/MoverNew/data') # rMBP
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Pipeline

In [4]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

# %%viztracer
active_data_mode_name = 'kdiba'
active_data_mode_registered_class = active_data_session_types_registered_classes_dict[active_data_mode_name]
active_data_mode_type_properties = known_data_session_type_properties_dict[active_data_mode_name]

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

## Animal `gor01`:
local_session_parent_path = local_session_root_parent_path.joinpath('gor01', 'one')
local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=['PhoHelpers', 'Spike3D-Minimal-Test', 'Unused'])

# local_session_parent_path = local_session_root_parent_path.joinpath('gor01', 'two')
# local_session_paths_list, local_session_names_list =  _find_local_session_paths(local_session_parent_path, blacklist=[])
    
# ### Animal `vvp01`:
# local_session_parent_path = local_session_root_parent_path.joinpath('vvp01', 'one')
# local_session_paths_list, local_session_names_list =  _find_local_session_paths(local_session_parent_path, blacklist=[])

# local_session_parent_path = local_session_root_parent_path.joinpath('vvp01', 'two')
# local_session_paths_list, local_session_names_list =  _find_local_session_paths(local_session_parent_path, blacklist=[])

# local_session_paths_list = [local_session_parent_path.joinpath(a_name).resolve() for a_name in local_session_names_list]
basedir = local_session_paths_list[1] # NOT 3
print(f'basedir: {str(basedir)}')


# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
curr_active_pipeline = NeuropyPipeline.try_init_from_saved_pickle_or_reload_if_needed(active_data_mode_name, active_data_mode_type_properties, override_basepath=Path(basedir), override_post_load_functions=[], force_reload=False, active_pickle_filename='loadedSessPickle.pkl', skip_save=True)
active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess, epoch_name_whitelist=['maze1','maze2','maze']) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)
# active_session_filter_configurations = active_data_mode_registered_class.build_filters_pyramidal_epochs(sess=curr_active_pipeline.sess, epoch_name_whitelist=['maze','maze1','maze2'])
# active_session_filter_configurations = active_data_mode_registered_class.build_filters_pyramidal_epochs(sess=curr_active_pipeline.sess, epoch_name_whitelist=['maze1','maze2'])
# active_session_filter_configurations = active_data_mode_registered_class.build_filters_pyramidal_epochs(sess=curr_active_pipeline.sess, epoch_name_whitelist=['maze1'])
# active_session_filter_configurations = active_data_mode_registered_class.build_filters_pyramidal_epochs(sess=curr_active_pipeline.sess, epoch_name_whitelist=['maze2'])
print(f'active_session_filter_configurations: {active_session_filter_configurations}')

# sess.epochs.t_start = 22.26 # exclude the first short period where the animal isn't on the maze yet
# active_session_filter_configurations = {'maze1': lambda x: (x.filtered_by_epoch(x.epochs.get_named_timerange('maze1')), x.epochs.get_named_timerange('maze1')),
#     'maze2': lambda x: (x.filtered_by_epoch(x.epochs.get_named_timerange('maze2')), x.epochs.get_named_timerange('maze2')),
#     'maze': lambda x: (x.filtered_by_epoch(NamedTimerange(name='maze', start_end_times=[x.epochs['maze1'][0], x.epochs['maze2'][1]])), NamedTimerange(name='maze', start_end_times=[x.epochs['maze1'][0], x.epochs['maze2'][1]]))
# }

active_session_computation_configs = active_data_mode_registered_class.build_default_computation_configs(sess=curr_active_pipeline.sess, time_bin_size=0.03333) #1.0/30.0 # decode at 30fps to match the position sampling frequency
# active_session_computation_configs[0].pf_params.smooth = (0.0, 0.0)
# active_session_computation_configs = build_eloy_computation_configs(sess=curr_active_pipeline.sess)
curr_active_pipeline.filter_sessions(active_session_filter_configurations) ## NOTE: BUG: TODO: this fails when the pipeline is loaded if the active_session_filter_configurations were initially defined in the notebook itself (because it can't get the source code).

# Whitelist Mode:
computation_functions_name_whitelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                     ]  # '_perform_pf_find_ratemap_peaks_peak_prominence2d_computation'
computation_functions_name_blacklist=None

# # Blacklist Mode:
# computation_functions_name_whitelist=None
# computation_functions_name_blacklist=['_perform_spike_burst_detection_computation','_perform_recursive_latent_placefield_decoding']

curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_whitelist=computation_functions_name_whitelist, computation_functions_name_blacklist=computation_functions_name_blacklist, fail_on_exception=True, debug_print=False) #, overwrite_extant_results=False  ], fail_on_exception=True, debug_print=False)

# curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_blacklist=['_perform_spike_burst_detection_computation'], debug_print=False, fail_on_exception=False) # whitelist: ['_perform_baseline_placefield_computation']

curr_active_pipeline.prepare_for_display(root_output_dir=global_data_root_parent_path.joinpath('Output'), should_smooth_maze=True) # TODO: pass a display config

curr_active_pipeline.save_pipeline()

local_session_names_list: ['2006-6-13_14-42-6', '2006-6-07_11-26-53', '2006-6-08_14-26-15', '2006-6-09_3-23-37', '2021-12-06 Code Backup', '2006-6-09_1-22-43', 'DibaCode', 'KamranMatlabCode', '2006-6-12_15-55-31']
basedir: /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53
finalized_loaded_sess_pickle_path: /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl
Loading loaded session pickle file results to /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl... done.
Loading pickled pipeline success: /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl.
property already present in pickled version. No need to save.
global_named_timerange: <NamedTimerange: {'name': 'maze', 'start_end_times': [0.0, 1932.4200048116618]};>, first_included_epoch_name: maze1, last_included_epoch_name: maze2
active_session_filter_configurations: {'maze1': <function DataSessionFormatBaseRegisteredClass.build_default_filter_functions.<locals>.<dictcomp>.<lam

PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl')

## 🗨️🟢 2022-10-26 - Jonathan Firing Rate Analyses

In [5]:
# ==================================================================================================================== #
# Perform missing global computations                                                                                  #
# ==================================================================================================================== #
curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_jonathan_replay_firing_rate_analyses', '_perform_short_long_pf_overlap_analyses'], fail_on_exception=True, debug_print=True)

## Get global 'jonathan_firing_rate_analysis' results:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis['neuron_replay_stats_df'], curr_jonathan_firing_rate_analysis['rdf']['rdf'], curr_jonathan_firing_rate_analysis['rdf']['aclu_to_idx'], curr_jonathan_firing_rate_analysis['irdf']['irdf']
neuron_replay_stats_df

global_computation_results is None. Building initial global_computation_results...
Performing _execute_computation_functions(...) with 2 registered_computation_functions...
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
replays_df: None, type(replays_df): <class 'NoneType'>
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
_execute_computation_functions(...): 
	accumulated_errors: None


,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays
2,-28.309544,True,5.460280,True,False,SplitPartitionMembership.SHARED,1.259629,0.930088,-0.329541,7.888768,8.177310,0.288543,4.574199,4.553699,-0.020499,0,130,109,21
3,69.888344,True,77.220275,True,False,SplitPartitionMembership.SHARED,1.066982,1.261272,0.194290,9.671088,8.505168,-1.165920,5.369035,4.883220,-0.485815,1,136,122,14
4,20.789400,True,50.782382,True,False,SplitPartitionMembership.SHARED,0.632984,0.600613,-0.032371,6.796109,8.598054,1.801945,3.714546,4.599333,0.884787,2,60,45,15
5,-5.648493,True,111.211852,True,False,SplitPartitionMembership.SHARED,1.706913,1.655163,-0.051749,9.894685,10.845931,0.951246,5.800799,6.250547,0.449748,3,230,190,40
6,32.119926,True,58.336066,True,False,SplitPartitionMembership.SHARED,33.163081,29.304612,-3.858469,119.227245,103.406793,-15.820452,76.195163,66.355702,-9.839460,4,486,396,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,-77.408488,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.815471,1.205806,0.390335,9.817662,17.696734,7.879071,5.316566,9.451270,4.134703,59,44,33,11
62,-32.086386,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.496043,0.999683,0.503639,7.474688,7.575587,0.100899,3.985365,4.287635,0.302269,60,93,72,21
63,73.665186,True,65.889750,True,False,SplitPartitionMembership.SHARED,2.334267,3.244755,0.910488,14.049291,17.298017,3.248726,8.191779,10.271386,2.079607,61,145,97,48
64,20.789400,True,20.567647,True,False,SplitPartitionMembership.SHARED,1.233777,1.993639,0.759862,15.434507,17.786444,2.351938,8.334142,9.890042,1.555900,62,174,150,24


## 🗨️🟢 2022-11-05 - Pho-Jonathan Batch Outputs of Firing Rate Figures

In [6]:
# ==================================================================================================================== #
# Batch Output of Figures                                                                                              #
# ==================================================================================================================== #

# %matplotlib qt

from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
short_only_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY]
short_only_aclus = short_only_df.index.values.tolist()
long_only_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY]
long_only_aclus = long_only_df.index.values.tolist()
shared_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.SHARED]
shared_aclus = shared_df.index.values.tolist()
print(f'shared_aclus: {shared_aclus}')
print(f'long_only_aclus: {long_only_aclus}')
print(f'short_only_aclus: {short_only_aclus}')

active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63, 64, 65]
long_only_aclus: [17, 18, 21, 24, 61, 62]
short_only_aclus: []


In [7]:
# ==================================================================================================================== #
# Output Figures to File                                                                                               #
# ==================================================================================================================== #
## PDF Output
# %matplotlib qtagg
import matplotlib
# configure backend here
# matplotlib.use('Qt5Agg')
# backend_qt5agg
matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

n_max_page_rows = 10
_batch_plot_kwargs_list = _build_batch_plot_kwargs(long_only_aclus, short_only_aclus, shared_aclus, active_identifying_session_ctx, n_max_page_rows=n_max_page_rows)
active_out_figures_list = _perform_batch_plot(curr_active_pipeline, _batch_plot_kwargs_list, figures_parent_out_path=None, write_pdf=False, write_png=True, progress_print=True, debug_print=False)

include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze


/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/DisplayFunctions/MultiContextComparingDisplayFunctions/MultiContextComparingDisplayFunctions.py:1003: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved /home/halechr/repos/PhoPy3DPositionAnalysis2021/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2022-12-07/kdiba_2006-6-07_11-26-53_batch_plot_test_long_only_[17, 18, 21, 24, 61, 62].png
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze


/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/DisplayFunctions/MultiContextComparingDisplayFunctions/MultiContextComparingDisplayFunctions.py:1003: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved /home/halechr/repos/PhoPy3DPositionAnalysis2021/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2022-12-07/kdiba_2006-6-07_11-26-53_batch_plot_test_shared_1of6_[2, 3, 4, 5, 6, 7, 8, 9, 10, 11].png
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze


...

	 saved /home/halechr/repos/PhoPy3DPositionAnalysis2021/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2022-12-07/kdiba_2006-6-07_11-26-53_batch_plot_test_shared_4of6_[36, 37, 38, 39, 40, 41, 42, 43, 44, 45].png
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze


/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/DisplayFunctions/MultiContextComparingDisplayFunctions/MultiContextComparingDisplayFunctions.py:1003: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved /home/halechr/repos/PhoPy3DPositionAnalysis2021/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2022-12-07/kdiba_2006-6-07_11-26-53_batch_plot_test_shared_5of6_[46, 47, 48, 49, 50, 51, 52, 53, 54, 55].png
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze


/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/DisplayFunctions/MultiContextComparingDisplayFunctions/MultiContextComparingDisplayFunctions.py:1003: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()


	 saved /home/halechr/repos/PhoPy3DPositionAnalysis2021/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2022-12-07/kdiba_2006-6-07_11-26-53_batch_plot_test_shared_6of6_[56, 57, 58, 59, 60, 63, 64, 65].png


# Individual Plotting Tests:

### Common Config

In [11]:
# display matplotlib figures in a qt backed window. Works in Jupyter notebooks
from neuropy.utils.matplotlib_helpers import enumTuningMap2DPlotVariables # for getting the variant name from the dict

from pyphocorehelpers.plotting.figure_management import PhoActiveFigureManager2D, capture_new_figures_decorator
fig_man = PhoActiveFigureManager2D(name=f'fig_man') # Initialize a new figure manager
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.DockAreaWrapper import DockAreaWrapper


## PDF Output, NOTE this is single plot stuff: uses active_config_name
from matplotlib.backends import backend_pdf
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, build_pdf_metadata_from_display_context, programmatic_display_to_PDF

# # ## OPTIONS/SETTINGS:
# # enable_saving_to_disk = True
# # # enable_saving_to_disk = False
# # # open_folders_in_system_teafile_manager = True
# # open_folders_in_system_file_manager = False

# ## Old Individual Plotting Function Configs, still used as of 2022-08-10:
# # figure_format_config = dict(subplots=(20, 8), max_screen_figure_size=(2256, 2048)) # bad way
# # figure_format_config = dict(subplots=(None, 8), max_screen_figure_size=(None, 1868)) # good way
# figure_format_config = dict(subplots=(None, 8), max_screen_figure_size=(None, None)) # good way
# # figure_format_config = dict(subplots=(20, 8), max_screen_figure_size=(None, 1868))

# # figure_format_config = dict(subplots=(6, 6), max_screen_figure_size=(2256, 2048)) # bad way
# # RowColTuple(subplots[0], subplots[1])

# ## Add explicit column/row widths to fix window sizing issue:
# # figure_format_config = (dict(fig_column_width=1.0, fig_row_height=1.0) | figure_format_config)
# figure_format_config = (dict(fig_column_width=2.0, fig_row_height=2.0) | figure_format_config)
# # figure_format_config = (dict(fig_column_width=5.0, fig_row_height=5.0) | figure_format_config)

# ## Add additional default options:
# figure_format_config = (dict(enable_spike_overlay=False, debug_print=True, enable_saving_to_disk=enable_saving_to_disk) | figure_format_config)
# figure_format_config

# NOTE: if you want the outputs to be rendered to the same PDF, you must wrap all the plotting in the backend_pdf context:
def wrap_display_function_to_save_pdf(curr_display_function_name='_display_normal', filter_name='maze'):
    """ implicitly captures: active_identifying_filtered_session_ctx, _get_curr_figure_format_config 
    
    Usage:
        wrap_display_function_to_save_pdf(curr_display_function_name='_display_normal', filter_name=active_config_name)
    """
    built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name) # but this is specific to a given display function, right?
    active_identifying_display_ctx = active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name=curr_display_function_name)
    figure_format_config = _get_curr_figure_format_config() # Fetch the context from the GUI
    figure_format_config['enable_saving_to_disk'] = False # don't use the in-built figure export/saving to disk functionality as we want to wrap the output figure with the Pdf saving, not write to a .png
    with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
        # set_rasterized,  Axes.set_rasterization_zorder
        # fig0, figList1 = curr_active_pipeline.display(curr_display_function_name, filter_name, debug_print=False, enable_saving_to_disk=enable_saving_to_disk)
        ## TODO: main display function call done here:
        out_figures_list = curr_active_pipeline.display(curr_display_function_name, filter_name, **figure_format_config)
        if not isinstance(out_figures_list, (list, tuple)):
            # not a list, just a scalar object
            plots = [out_figures_list] # make a single-element list
        else:
            # it is a list
            if len(out_figures_list) == 2:
                fig0, figList1 = out_figures_list # unpack
                plots = [fig0, *figList1]
            else:
                # otherwise just try and set the plots to the list
                plots = out_figures_list
                
        for i, a_fig in enumerate(plots):
            pdf.savefig(a_fig, transparent=True)
            pdf.attach_note(f'Page {i + 1}') 

            

## Single (Session, Filter) Context Plotting:

### Utility:

In [ ]:
# Reload display functions:
curr_active_pipeline.reload_default_display_functions()
curr_active_pipeline.registered_display_function_names # ['_display_1d_placefield_validations', '_display_2d_placefield_result_plot_ratemaps_2D', '_display_2d_placefield_result_plot_raw', '_display_normal', '_display_placemaps_pyqtplot_2D', '_display_decoder_result', '_display_plot_most_likely_position_comparisons', '_display_two_step_decoder_prediction_error_2D', '_display_two_step_decoder_prediction_error_animated_2D', '_display_spike_rasters_pyqtplot_2D', '_display_spike_rasters_pyqtplot_3D', '_display_spike_rasters_pyqtplot_3D_with_2D_controls', '_display_spike_rasters_vedo_3D', '_display_spike_rasters_vedo_3D_with_2D_controls', '_display_spike_rasters_window', '_display_speed_vs_PFoverlapDensity_plots', '_display_3d_image_plotter', '_display_3d_interactive_custom_data_explorer', '_display_3d_interactive_spike_and_behavior_browser', '_display_3d_interactive_tuning_curves_plotter']
print(curr_active_pipeline.registered_display_function_names)

In [ ]:
## Test getting figure save paths:
_test_fig_path = curr_active_config.plotting_config.get_figure_save_path('test')
print(f'_test_fig_path: {_test_fig_path}\n\t exists? {_test_fig_path.exists()}')

In [9]:
%matplotlib --list 
# Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'gtk4', 'wx', 'qt4', 'qt5', 'qt6', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']

Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'gtk4', 'wx', 'qt4', 'qt5', 'qt6', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']


In [10]:
%matplotlib qt
## NOTE THAT ONCE THIS IS SET TO qt, it cannot be undone!

In [ ]:
fig_man.close_all()

In [ ]:
extant_figs_dict = fig_man.figures_dict
extant_fig_nums = fig_man.figure_nums
fig_num_figure_still_exists = [plt.fignum_exists(a_fig_num) for a_fig_num in extant_fig_nums]
fig_num_figure_still_exists #     # Figure is still opened or is closed
# if plt.fignum_exists(<figure number>):
#     # Figure is still opened
# else:
#     # Figure is closed
extant_figs_dict

# mpl.rcParams['toolbar'] = 'None'

In [ ]:
fig_man.reshow_figure(extant_figs_dict[10])

## Systematic Display Function Testing

### Matplotlib-based plots:

In [12]:
import matplotlib
# matplotlib.use('AGG') # non-interactive backend
# %matplotlib -l

matplotlib.use('Qt5Agg') # non-interactive backend
## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

curr_active_pipeline.filtered_session_names # ['maze', 'sprinkle']
active_config_name = 'maze'

# active_display_to_pdf_fn = wrap_display_function_to_save_pdf
active_display_to_pdf_fn = programmatic_display_to_PDF

In [ ]:
# Crashed the kernel last time I tried it:
# %%capture
# active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefield_validations', filter_name=active_config_name) # 🟢✅ Working

In [13]:
# %%capture
# active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_raw', filter_name=curr_active_pipeline.filtered_contexts.maze) # 🟢✅ Working
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_raw') # 🟢✅ Working

In [14]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_ratemaps_2D') # , filter_name=active_config_name # 🟢 Mostly Working # works but now only displays a single filter (where before it displayed all of them and rendered them to individual pages of the PDF)
# BROKEN: TypeError: neuropy.utils.debug_helpers.safely_accepts_kwargs.<locals>._safe_kwargs_fn() got multiple values for keyword argument 'computation_config'

/home/halechr/miniconda3/envs/PhoLabVizClone/lib/python3.9/site-packages/ipykernel/eventloops.py:107: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  app.exec_()


In [15]:
%%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_normal', filter_name=active_config_name) # 🟢✅ Working

ValueError: too many values to unpack (expected 2)

## 🐞👁️‍🗨️🔜 TODO: FINISH THIS UP AND FIGURE OUT WHATEVER THE HELL I'M DOING HERE

In [16]:
curr_display_function_name = '_display_2d_placefield_result_plot_ratemaps_2D'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
out_fig_list = []
active_identifying_display_ctx = active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name=curr_display_function_name)
figure_format_config = _get_curr_figure_format_config() # Fetch the context from the GUI
figure_format_config['enable_saving_to_disk'] = False # don't use the in-built figure export/saving to disk functionality as we want to wrap the output figure with the Pdf saving, not write to a .png
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    ## TypeError: neuropy.utils.debug_helpers.safely_accepts_kwargs.<locals>._safe_kwargs_fn() got multiple values for keyword argument 'computation_config'
    for filter_name in curr_active_pipeline.filtered_session_names:
        print(f'filter_name: {filter_name}')
        active_identifying_ctx = active_identifying_display_ctx.adding_context('plot_variable', variable_name=enumTuningMap2DPlotVariables.SPIKES_MAPS)
        active_identifying_ctx_string = active_identifying_ctx.get_description(separator='|') # Get final discription string
        out_fig_list.extend(curr_active_pipeline.display(curr_display_function_name, filter_name, plot_variable=enumTuningMap2DPlotVariables.SPIKES_MAPS, fignum=active_identifying_ctx_string, **figure_format_config)) # works!
        active_identifying_ctx = active_identifying_display_ctx.adding_context('plot_variable', variable_name=enumTuningMap2DPlotVariables.TUNING_MAPS)
        active_identifying_ctx_string = active_identifying_ctx.get_description(separator='|') # Get final discription string
        out_fig_list.extend(curr_active_pipeline.display(curr_display_function_name, filter_name, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, fignum=active_identifying_ctx_string, **figure_format_config))
        for a_fig in out_fig_list:
            pdf.savefig(a_fig, transparent=True)

NameError: name '_build_pdf_pages_output_info' is not defined

In [17]:
%%capture
curr_display_function_name = '_display_decoder_result'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    plots = curr_active_pipeline.display(curr_display_function_name, filter_name)
    print(plots)
    # pdf.savefig(a_fig)

NameError: name '_build_pdf_pages_output_info' is not defined

### PyQtGraph-based plots:

#### Decoder Plots:

In [ ]:
# Must switch back to the interactive backend here for the interactive/animated decoder plots:
matplotlib.use('Qt5Agg')
# backend_qt5agg
import matplotlib.pyplot as plt
# plt.switch_backend('Qt5Agg')

# ## MATPLOTLIB Imports:
# import matplotlib
# # configure backend here
# matplotlib.use('Qt5Agg')
# import matplotlib.pyplot as plt
# import matplotlib as mpl

## PDF Output, NOTE this is single plot stuff: uses active_config_name
from matplotlib.backends import backend_pdf
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, build_pdf_metadata_from_display_context, programmatic_display_to_PDF

In [ ]:
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_animated_2D', active_config_name, variable_name='p_x_given_n')

In [ ]:
## This plot looks phenominal, and the slider works!
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_2D', active_config_name, variable_name='p_x_given_n') # NOW: TypeError: _temp_debug_two_step_plots_animated_imshow() missing 1 required positional argument: 'time_binned_position_df'

In [ ]:
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_2D', active_config_name, variable_name='p_x_given_n_and_x_prev')  # this one doesn't work!

In [ ]:
## 2022-10-04 Modern Programmatic PDF outputs:
# programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_plot_decoded_epoch_slices',  debug_print=False)
programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_plot_decoded_epoch_slices', filter_epochs='ripple', decoding_time_bin_size=0.02, debug_test_max_num_slices=128, debug_print=True)

In [ ]:
programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_plot_decoded_epoch_slices', filter_epochs='laps', debug_test_max_num_slices=128, debug_print=False)

In [ ]:
## TODO: trying to hack up 'replay's which aren't working because of the overlaps, and strangely my overlaps code doesn't seem to work

### 🔜 2022-08-10 👁️‍🗨️ NOW: Plot animal positions on the computed posteriors:
The process of plotting the animal position on the decoder plot needs to be refined. Currently it works by re-implementing 

🔜 NEXT STEP: TODO: Make a "Datasource" like approach perhaps to provide the actual animal position at each point in time?
🐞🔜 BUG TODO: Noticed that for Bapun Day5 data, it looks like the current position point is being plotted incorrectly (it doesn't even move across the space much)

In [ ]:
curr_display_function_name = '_display_spike_rasters_pyqtplot_2D'
curr_active_pipeline.display(curr_display_function_name, filter_name, debug_print=False, enable_saving_to_disk=enable_saving_to_disk) 

In [ ]:
## Works, displays my velocity/density result for both 2D and 1D:
# out_plot_1D, out_plot_2D = curr_active_pipeline.display('_display_speed_vs_PFoverlapDensity_plots', active_config_name)
curr_display_function_name = '_display_speed_vs_PFoverlapDensity_plots'
plots = curr_active_pipeline.display(curr_display_function_name, filter_name)
plots

In [ ]:
curr_display_function_name = '_display_placemaps_pyqtplot_2D'
out_plots = curr_active_pipeline.display(curr_display_function_name, filter_name, max_num_columns=8)    
out_plots[1].show()

In [ ]:
## KNOWN BAD, locks up, do not execute:
curr_display_function_name = 'display_firing_rate_trends'
plots = curr_active_pipeline.display(curr_display_function_name, filter_name)

In [ ]:
# a_plot = plots[0] # PlotWidget 
# a_plot_item = a_plot.plotItem # PlotItem
# a_plot.scene() # GraphicsScene
export_pyqtgraph_plot(plots[0])

# Non-batch/Automated

### 3D (PyVista/Vedo/etc)-based plots:

In [ ]:
curr_active_pipeline.display('_display_3d_interactive_spike_and_behavior_browser', active_config_name) # this works now!

In [ ]:
display_dict = curr_active_pipeline.display('_display_3d_interactive_custom_data_explorer', active_config_name) # does not work, missing color info?
iplapsDataExplorer = display_dict['iplapsDataExplorer']
# plotter is available at
p = display_dict['plotter']
iplapsDataExplorer

In [ ]:
# curr_kdiba_pipeline.display(DefaultDisplayFunctions._display_3d_interactive_custom_data_explorer, 'maze1') # works!
curr_active_pipeline.display('_display_3d_interactive_tuning_curves_plotter', 'maze1_PYR') # works!